In [1]:
import requests
import pandas as pd
from datetime import datetime
import calendar

In [2]:
API_KEY = "cba37f05704650d3ce454be815d62ddb-57583e59ae1ce78ff01e204743ce69d0"  # Replace with your OANDA API key
ACCOUNT_ID = "101-004-30016702-001"  # Replace with your OANDA account ID
OANDA_API_URL = "https://api-fxpractice.oanda.com/v3"  # Demo API URL

In [3]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}

## Getting Account and Currencies Information 

In [7]:
def get_account_balance():
    url = f"{OANDA_API_URL}/accounts/{ACCOUNT_ID}/summary"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        account_data = response.json()["account"]
        print(f"Balance: {account_data['balance']} £")
        print(f"Unrealized P/L: {account_data['unrealizedPL']}")
        print(f"Margin Used: {account_data['marginUsed']}")
    else:
        print("Error fetching account details:", response.json())

In [8]:
def get_available_currencies() -> list:
    url = f"{OANDA_API_URL}/accounts/{ACCOUNT_ID}/instruments"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        instruments = response.json()["instruments"]
        currencies = [instrument['name'] for instrument in instruments if instrument['type'] == "CURRENCY"]
        return currencies
    else:
        print("Error fetching instruments:", response.json())
        return []

In [9]:
def get_currency_prices(instruments):
    # Create a comma-separated string of instruments
    instruments_str = ",".join(instruments)
    url = f"{OANDA_API_URL}/accounts/{ACCOUNT_ID}/pricing"
    params = {"instruments": instruments_str}  # Passing the instruments as query parameters

    response = requests.get(url, headers=headers, params=params)  # Using GET method here
    if response.status_code == 200:
        prices_data = response.json()["prices"]
        for price in prices_data:
            instrument = price["instrument"]
            bid = price["bids"][0]["price"]
            ask = price["asks"][0]["price"]
            print(f"{instrument}: Bid = {bid}, Ask = {ask}")
    else:
        print("Error fetching prices:", response.json())

In [10]:
if __name__ == "__main__":
    print("Fetching Account Balance:")
    get_account_balance()

    print("\nFetching List of Available Currency Pairs:")
    currency_pairs = get_available_currencies()
    if currency_pairs:
        print(f"Found {len(currency_pairs)} currency pairs.")
        print("\nFetching Bid and Ask Prices:")
        get_currency_prices(currency_pairs)
    else:
        print("No currency pairs found.")

Fetching Account Balance:
Balance: 100000.0000 £
Unrealized P/L: 0.0000
Margin Used: 0.0000

Fetching List of Available Currency Pairs:
Found 68 currency pairs.

Fetching Bid and Ask Prices:
EUR_HUF: Bid = 399.483, Ask = 400.785
HKD_JPY: Bid = 19.24027, Ask = 19.24227
EUR_PLN: Bid = 4.30280, Ask = 4.30730
GBP_SGD: Bid = 1.70851, Ask = 1.71166
USD_SEK: Bid = 10.51485, Ask = 10.52375
EUR_DKK: Bid = 7.45766, Ask = 7.45896
EUR_CAD: Bid = 1.49925, Ask = 1.50015
USD_HUF: Bid = 367.520, Ask = 368.950
AUD_CAD: Bid = 0.92490, Ask = 0.92598
USD_MXN: Bid = 19.86860, Ask = 19.89460
GBP_USD: Bid = 1.30475, Ask = 1.30544
CAD_SGD: Bid = 0.94897, Ask = 0.94996
NZD_SGD: Bid = 0.79492, Ask = 0.79607
ZAR_JPY: Bid = 8.474, Ask = 8.499
SGD_JPY: Bid = 114.051, Ask = 114.184
GBP_ZAR: Bid = 22.95550, Ask = 23.02250
USD_JPY: Bid = 149.511, Ask = 149.555
EUR_TRY: Bid = 37.21690, Ask = 37.33500
EUR_JPY: Bid = 162.473, Ask = 162.547
AUD_SGD: Bid = 0.87819, Ask = 0.87903
EUR_NZD: Bid = 1.78925, Ask = 1.79032
GBP_H

## Gathering Historical Forex Data

In [4]:
def get_historical_data(instrument="EUR_USD", start_date=None, end_date=None, granularity="D"):
    url = f"{OANDA_API_URL}/instruments/{instrument}/candles"
    
    params = {
        "from": start_date,
        "to": end_date,
        "granularity": granularity,
        "price": "BA"  # Fetch both bid and ask prices
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        candles = response.json().get("candles", [])
        data = []
        for candle in candles:
            time = candle["time"]
            bid_open = candle["bid"]["o"]
            bid_close = candle["bid"]["c"]
            ask_open = candle["ask"]["o"]
            ask_close = candle["ask"]["c"]
            data.append([time, bid_open, bid_close, ask_open, ask_close])
        return data
    else:
        print("Error fetching historical data:", response.json())
        return []

# Function to iterate through 2010 to 2017 and fetch monthly data
def fetch_data_for_period(instrument="EUR_USD", start_year=2010, end_year=2017):
    all_data = []

    current_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)

    while current_date < end_date:
        # Calculate the last day of the current month
        last_day_of_month = calendar.monthrange(current_date.year, current_date.month)[1]
        month_end = current_date.replace(day=last_day_of_month)

        # Set start and end date strings in the correct format
        start_date_str = current_date.strftime('%Y-%m-%dT%H:%M:%SZ')
        end_date_str = month_end.strftime('%Y-%m-%dT%H:%M:%SZ')

        print(f"Fetching data from {start_date_str} to {end_date_str}...")
        
        # Fetch data for the current month
        data = get_historical_data(instrument=instrument, start_date=start_date_str, end_date=end_date_str, granularity="D")
        
        if data:
            all_data.extend(data)
        
        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1, day=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1, day=1)

    # Save data to a CSV file
    columns = ["Time", "Bid_Open", "Bid_Close", "Ask_Open", "Ask_Close"]
    df = pd.DataFrame(all_data, columns=columns)
    df.to_csv(f"HistoricData/{instrument}_2010_2017_bid_ask.csv", index=False)
    print(f"Data saved to {instrument}_2010_2017_bid_ask.csv")

In [6]:
if __name__ == "__main__":
    currencies = "EUR_GBP"
    print(f"Fetching Historical Bid and Ask Prices for {currencies} (2010 - 2017):")
    fetch_data_for_period(instrument=currencies, start_year=2010, end_year=2017)

Fetching Historical Bid and Ask Prices for EUR_GBP (2010 - 2017):
Fetching data from 2010-01-01T00:00:00Z to 2010-01-31T00:00:00Z...
Fetching data from 2010-02-01T00:00:00Z to 2010-02-28T00:00:00Z...
Fetching data from 2010-03-01T00:00:00Z to 2010-03-31T00:00:00Z...
Fetching data from 2010-04-01T00:00:00Z to 2010-04-30T00:00:00Z...
Fetching data from 2010-05-01T00:00:00Z to 2010-05-31T00:00:00Z...
Fetching data from 2010-06-01T00:00:00Z to 2010-06-30T00:00:00Z...
Fetching data from 2010-07-01T00:00:00Z to 2010-07-31T00:00:00Z...
Fetching data from 2010-08-01T00:00:00Z to 2010-08-31T00:00:00Z...
Fetching data from 2010-09-01T00:00:00Z to 2010-09-30T00:00:00Z...
Fetching data from 2010-10-01T00:00:00Z to 2010-10-31T00:00:00Z...
Fetching data from 2010-11-01T00:00:00Z to 2010-11-30T00:00:00Z...
Fetching data from 2010-12-01T00:00:00Z to 2010-12-31T00:00:00Z...
Fetching data from 2011-01-01T00:00:00Z to 2011-01-31T00:00:00Z...
Fetching data from 2011-02-01T00:00:00Z to 2011-02-28T00:00:00Z